In [1]:
import math
import copy
from task import Task
from caseLoader import CaseLoader
from taskType import TaskType
import random

In [2]:
MT = 10 # microtick

In [3]:
loader = CaseLoader()
cases = loader.load_test_case("inf_30_30", 0)
cases = [c for c in cases if c.type == TaskType.TIME]

Searching for test cases at:
./test_cases/inf_30_30
Loaded case_group: inf_30_30 - case: 0 test case(s).


In [4]:
def get_ready(task_set, cycle, ready_list):
    for task in task_set:
        if cycle % task.period == 0:
            new_job = copy.deepcopy(task)
            new_job.deadline = new_job.deadline + cycle
            new_job.release_time = cycle
            ready_list.append(new_job)
    
    # sort on deadline
    # https://www.geeksforgeeks.org/sorting-objects-of-user-defined-class-in-python/
    ready_list = sorted(ready_list, key=lambda t: t.deadline) 
    
    return sorted(ready_list, key=lambda t: t.deadline)
        

# ts is set of TT tasks
def edf(ts):
    T = math.lcm(2000,3000,4000) # least common multiple of TT task periods. 2000 mt = 
    S = [] # schedule will be hyperperiod long. 12 000 microticks == 120 000 microsecs == 120 ms
    ready_list = []
    wcrts = {} # worst case response times

    for t in range(0, T*2):
        ready_list = get_ready(ts, t, ready_list)
        
        for task in ready_list:
            if task.duration > 0 and task.deadline <= t:
                return []
        
            # job done check response time gt wcrt and remove from ready list
            if task.duration == 0 and task.deadline >= t:
                response_time = t - task.release_time
                
                if task.name not in wcrts or response_time >= wcrts[task.name]:
                    wcrts[task.name] = response_time
                
                ready_list.remove(task)
                
            
        if ready_list == []:
            S.append("IDLE")
            continue
        else:
            # EDF get next job to execute 
            S.append(ready_list[0].name)
            ready_list[0].duration = ready_list[0].duration - 1
    
    if ready_list != []:
        return [], []
    
    return S, wcrts

In [5]:
s, wcrts = edf(cases)

In [6]:
print(wcrts)

{'tTT0': 16, 'tTT1': 33, 'tTT4': 39, 'tTT5': 42, 'tTT6': 63, 'tTT11': 81, 'tTT14': 85, 'tTT18': 113, 'tTT21': 125, 'tTT24': 156, 'tTT26': 162, 'tTT27': 171, 'tTT7': 210, 'tTT8': 235, 'tTT9': 286, 'tTT10': 322, 'tTT17': 380, 'tTT19': 439, 'tTT20': 444, 'tTT23': 592, 'tTT25': 594, 'tTT28': 674, 'tTT2': 703, 'tTT3': 704, 'tTT12': 835, 'tTT13': 842, 'tTT15': 859, 'tTT16': 874, 'tTT22': 880, 'tTT29': 884}


In [7]:
# problem given as a list of cities and their position given by cartesian coordinates

def swap(l, index_1, index_2):
    l_copy = l[:] # list passed as reference, work with a copy instead
    tmp = l_copy[index_1]
    l_copy[index_1] = l[index_2]
    l_copy[index_2] = tmp
    
    return l_copy


def cost_edge(x1, x2):
    return np.linalg.norm(np.array([x1[0] - x2[0], x1[1] - x2[1]]))

# assume element of l has form (ID, [x, y]). Cost is sum of lengths between points. 
def cost_tour(l):
    return sum([cost_edge(l[i][1], l[(i+1) % len(l)][1]) for i in range(len(l))])

rand = random.Random()

# let neighborhood be defined by the solutions that can be
# obtained by swapping two cities in current solution
def random_neighbor_solution(l):
    length = len(l)
    i = rand.randint(0, length-1)
    j = rand.randint(0, length-1)
    
    while i == j:
        j = rand.randint(0, length-1)
        
    return swap(l, i, j)

def p(delta, t):
    return np.exp( -delta / t )

# simulated annealing 
def sa(s0, t0, a, stopcriterion_sec, neighborhood_f = None, log_costs = False):
    # initialize variables
    sec0 = int(time.time())
    sec_now = int(time.time())
    s_best = s0[:]
    cost_best = cost_tour(s_best)
    s_cur = s0[:]
    cost_cur = cost_tour(s_cur)
    t = t0
    
    cost_log = [cost_cur]
    
    n_solutions = 0
    
    # does not matter if < or <= if 0 then new solution will be selected no matter what     
    while sec_now - sec0 < stopcriterion_sec:
        s_tmp = neighborhood_f(s_cur)
        cost_tmp = cost_tour(s_tmp)
        delta = cost_tmp - cost_cur
        
        # accept randomly drawn solution from current neighborhood if better or with some probability
        if delta <= 0 or p(delta, t) > rand.uniform(0.0, 1.0):
            s_cur = s_tmp
            cost_cur = cost_tmp
            
            if log_costs:
                cost_log.append(cost_cur)
            
            # keep track of the best solution 
            if cost_cur < cost_best:
                s_best = s_cur
                cost_best = cost_cur
                #print("new best cost: ", cost_best)
                #cost_log.append(cost_best)
        
        # update temperature 
        t = t * a
        
        n_solutions = n_solutions + 1
        
        # update sec
        sec_now = int(time.time())
    
    return (s_best, cost_best, n_solutions) if log_costs else (s_best, cost_best) 
    

In [18]:
#Schedulability of ET tasks under a given polling task

#Data:
#polling task budget: Cp
#polling task period: Tp
#polling task deadline: Dp
#subset of ET tasks to check Tau^ET (T_ET)
#pi = priority
#Ci = Computation time?
#ti = task period (tuple)

#result: small_tau (Boolean)

####

def unpack(task):
    return (task.priority, task.duration, task.period, task.deadline)

def calculate_schedulabiltiy(Tp, Dp, Cp, task_periods):
    #compute delta and alpha accordingly to [2]
    Delta = Tp + Dp - 2*Cp
    alpha = Cp/Tp
    
    #hyperperiod is lcm of all task periods in T_ET (all values must be from the chosen subset of ET tasks from the .csv)
    periods = []
    for task in task_periods:
        (pi, Ci, Ti, Di) = unpack(task)
        periods.append(Ti)

    hyperperiod = math.lcm(*periods)

    #loop
    for task_period in task_periods:
        (pi, Ci, Ti, Di) = unpack(task_period)
        t = 0
        #initialize the response time of ti (task period) to a value exceeding the deadline
        response_time = Di + 1

        #remember we are dealing with constrained deadline tasks for the AdvPoll, hence, in the worst case arrival pattern, the intersection must lie within the hyperperiod if the task is schedulable

        while t <= hyperperiod:
            #the supply at time t ([1])
            supply = alpha*(t-Delta)

            #compute the maximum demand at time t according to Eq. 2
            demand = 0
            for tj in task_periods:
                (pj, Cj, Tj, Dj) = unpack(tj)

                if pj >= pi:
                    demand  = demand + math.ceil(t/Tj)*Cj
            
            #According to lemma 1 of [1], we are searching for the earliest time, when the supply exceeds the demand
            if supply >= demand:
                response_time = t
                break
            
            t = t +1
        
            if response_time >= Di:
                return False
        
    return True


In [9]:
"""
Determine: You will have to design and implement an optimization algorithm that
determines an optimized solution which consists of the following: (i) The number of polling
servers, which then become extra TT tasks. (ii) For each polling server (task), the period,
budget, and deadline, as well as (iii) which sub-sets of ET tasks are handled within the
respective polling servers. Moreover, the TT tasks also need to be schedulable, i.e., given the
found polling tasks (and their parameters), it shall be possible to find (iv) a TT schedule
such that also the TT tasks are schedulable

"""


NUM_PS = 0
BUDGET = 1
PERIOD = 2
DEADLINE = 3
SUBSET = 4

# get a subset of pses from victim and delete these from victim
# -2 as to not steal all :)
def get_subset(s, victim_ps):
    num_et_tasks = rand.randint(0, len(victim_ps.et_subset)-2)
    new_ps_et_subset = []
        
    for i in range(num_et_tasks):
        new_ps_et_subset.append(victim_ps.et_subset[i])
        del victim_ps.et_subset[i]

    return new_ps_et_subset
    
# take polling servers as arg, 
def neighborhood(s, cycle):
    # select randomly a ps then randomly choose to change:
        # num_ps, period, budget, deadline or subset of et_tasks
        # et_tasks will have to take some from another/give 
        # do not worry whether newly generated solution is feasible 
        # return new solution 
    
    # num_ps, period, budget, deadline, subset
    
    # select ps
    ps = rand.randint(0, len(s)-1)
    
    parameter = rand.randint(0, 4)
    
    victim_ps_i = rand.randint(0,len(ps) - 1) # victim to change etc.
    sign = 1 if rand.randint(0,1) == 0 else -1
    
    if parameter == NUM_PS:
        new_ps_et_subset = get_subset(s, s[victim_ps_i])
        new_ps = Task("tTT" + str(cycle), 1000, 1000, TaskType.TIME, 7, 1000, new_ps_et_subset)
        
        return s + [new_ps]
    
    elif parameter == BUDGET:
        victim_ps.duration = ps.duration + sign * (cycle % 101)
    elif parameter == PERIOD:
        victim_ps.period = ps.period + sign * (cycle % 101)
    elif parameter == DEADLINE:
        victim_ps.deadline = ps.deadline + sign * (cycle % 101)
    elif parameter == SUBSET:
        other_ps_victim_i = rand.randint(0, len(ps) - 1)
        
        while other_ps_victim_i == victim_ps_i:
            other_ps_victim_i = rand.randint(0, len(ps) - 1)
            
        new_ps_et_subset = get_subset(s, s[other_victim_ps_i])
        
        s[victim_ps_i].et_subset += new_ps_et_subset
        
    return s
    
        
        

In [10]:
rand.randint(0,1)

0

In [11]:
def lars(a, b=None):
    return a

lars(1)

1

In [12]:
len(s)

24000

In [13]:
s1 = s[0:int(len(s)/2)]
s2 = s[int(len(s)/2):]

for i, j in zip(s1,s2):
    if s1 != s2:
        print("FAIL")

In [14]:
loader = CaseLoader()
ets = loader.load_test_case("inf_30_30", 0)
ets = [c for c in ets if c.type == TaskType.EVENT]

Searching for test cases at:
./test_cases/inf_30_30
Loaded case_group: inf_30_30 - case: 0 test case(s).


In [19]:
cnt = 0
for i in range(1, 5000):
    if calculate_schedulabiltiy(i,i,i, ets):
        cnt += 1

KeyboardInterrupt: 

In [ ]:
cnt